In [35]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output

# Charger vos données
data = pd.read_excel("../DB/All-at-once_DB.xlsx")

# Calcul des incertitudes absolues
data["Absolute Uncertainty"] = (data["1s uncertainty"] / 100) * data["Dose (Gy)"]

# Ajouter une colonne pour la combinaison de filtres
data['Filter Combo'] = data.apply(lambda row: f"{row['Particle']}_{row['Screen']}_{row['Code']}_{row['Case']}_{row['Thickness (cm)']}", axis=1)

output = Output()  # Créez une zone d'affichage pour la figure

# Initialiser une figure vide dans l'objet Output
fig = go.Figure()
fig.update_layout(
    title="Dose vs. Distance [log scales]",
    xaxis_title="Distance (m) [Log]",
    yaxis_title="Dose (Gy) [Log]",
    legend_title="Filter Combination",
    xaxis={'type': 'log'},
    yaxis={'type': 'log'},
    showlegend=True  # Assure que la légende est toujours affichée
)

display(output)  # Affiche l'objet Output une seule fois

# Fonction pour mettre à jour le graphique
def plot_filters(particle_types, screen_types, code_types, case_types, thickness_types):
    fig.data = []  # Efface les données précédentes de la figure
    filtered_data = data[data['Particle'].isin(particle_types) & 
                         data['Screen'].isin(screen_types) &
                         data['Code'].isin(code_types) &
                         data['Case'].isin(case_types) &
                         data['Thickness (cm)'].isin(thickness_types)]

    for combo in filtered_data['Filter Combo'].unique():
        df_subset = filtered_data[filtered_data['Filter Combo'] == combo]
        fig.add_trace(go.Scatter(x=df_subset["Distance (m)"], y=df_subset["Dose (Gy)"],
                                 mode='lines+markers', name=combo,
                                 line=dict(dash='dash'),
                                 error_y=dict(type='data', array=2*df_subset["Absolute Uncertainty"],
                                              visible=True)))
    with output:
        output.clear_output(wait=True)  # Efface la sortie précédente
        display(fig)  # Affichez la figure mise à jour

# Création des widgets
particle_selector = widgets.SelectMultiple(
    options=list(data['Particle'].unique()),
    value=[data['Particle'].unique()[0]],
    description='Particle:',
)

screen_selector = widgets.SelectMultiple(
    options=list(data['Screen'].unique()),
    value=[data['Screen'].unique()[0]],
    description='Screen:',
)

code_selector = widgets.SelectMultiple(
    options=list(data['Code'].unique()),
    value=[data['Code'].unique()[0]],
    description='Code:',
)

case_selector = widgets.SelectMultiple(
    options=list(data['Case'].unique()),
    value=[data['Case'].unique()[0]],
    description='Case:',
)

thickness_options = sorted(data['Thickness (cm)'].unique())
thickness_selector = widgets.SelectMultiple(
    options=thickness_options,
    value=[thickness_options[0]],
    description='Thickness:',
)

interactive_plot = widgets.interactive_output(plot_filters, {
    'particle_types': particle_selector, 
    'screen_types': screen_selector, 
    'code_types': code_selector,
    'case_types': case_selector,
    'thickness_types': thickness_selector
})

widget_box = widgets.VBox([
    particle_selector,
    screen_selector,
    code_selector,
    case_selector,
    thickness_selector
])

display(widget_box)


Output()